In [2]:
import numpy as np
import pandas as pd

import requests
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import base64


In [3]:
client_id = '7a2b21cebdb44bc282e6e415eaccf347'
client_secret = 'a276bff35be5492da4e8c8015ebcd5ae'

# Codifica las credenciales en base64
encoded = base64.b64encode(bytes(client_id + ':' + client_secret, 'utf-8')).decode('utf-8')

headers = {
    'Authorization': 'Basic ' + encoded
}

body = {
    'grant_type': 'client_credentials'
}

auth_response = requests.post('https://accounts.spotify.com/api/token', headers=headers, data=body)
access_token = auth_response.json()['access_token']

In [5]:
headers = {
    'Authorization': 'Bearer ' + access_token
}

artist_id = '55'

response = requests.get(f'https://api.spotify.com/v1/artists/{artist_id}/top-tracks?market=ES', headers=headers)
response.json()





IndentationError: unexpected indent (2073867630.py, line 8)